# Data Cleaning

In this section, I choose a subset of reviews to be considered for downstream data modeling. 


In [1]:
import pandas as pd
from matplotlib import pyplot as plt 
import numpy as np

In [ ]:
# choosing only the columns that are needed. The column 'Review' contains the text that makes the dataframe heavy. So it is dropped.
reviews = pd.read_csv('../data/reviews.csv', usecols=['AuthorId', 'ReviewId', 'RecipeId', 'Rating'])
# Data analysis shows 'Rating'=0 is not neccesarily bad rating 
# (Refer to https://www.kaggle.com/code/gemmin/sentiment-analysis)
# Removing 0 ratings
reviews = reviews[reviews['Rating']!=0]
# Only reviews by active reviewers (selected_authors) are accepted, as the model will be based on active reviewers.
# Active reviewer is whoever has more than a minimum number of reviews in the data
review_counts = reviews.groupby('AuthorId').count()
selected_authors = review_counts.loc[review_counts['ReviewId']>=10].index.values
reviews = reviews.loc[reviews['AuthorId'].isin(selected_authors)]

print('number of selected reviews:', reviews['ReviewId'].nunique())
print('number of selected reviewers:', reviews['AuthorId'].nunique())
print('number of selected recipes:', reviews['RecipeId'].nunique())
 